In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Joao\AppData\Local\Temp\ipykernel_6080\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
%%html
<style type='text/css'>
.CodeMirror{
    font-size: 15px;
}

div.output_area pre {
    font-size: 15px;
}
</style>

# Descrição

Este notebook gera uma tabela pra fezer testes no desenvolvimento do pacote usando o modelo gerado como o as pesquisas. A ideia é gerar uma tabela qua contenha uma coluna com  a variável alvo (chamada de "class" neste problema) e uma coluna com a os valores previsto pelo modelo (chamada de "prev_val", na tabela final)

Por meio desta tabela gerada podemos fazer testes no pacote desenvolvido e verificar se a previsão pelo modelo implemantado está correta.

In [25]:
import pandas as pd
import numpy as np
import joblib
from sklearn import metrics

## Abertura e tratamento dos dados

In [4]:
#Importa os dados
df = pd.read_csv("dados\\dados_teste.csv" ,sep=',');
df.head(3)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,57.0,F,148.0,54.0,33.6,78.0,135.0,27.4,18.8,16.0,137.0,C
1,30.0,M,179.7,75.7,22.8,83.0,142.0,37.6,8.0,50.0,223.0,C
2,25.0,F,160.0,57.4,17.5,72.0,127.0,34.9,31.0,51.0,200.0,A


In [14]:
#variáveis de previsao e alvo
var_prev=[var for var in df.columns if var != 'class'] 

X = df[var_prev]
y = df['class']

## Funções usadas no pipeline

Dúvida: esta função foii usada no pipeline, não sei o porque ela não ficou salva no binário do pipeline. Outras funções usadas com métodos do sklearn ficaram slavadas. Ver isto no futuro. Seria bom não precisar ficar reescrevendo funções, seria bom conseguir salver tudo em um único arquivo e importar este arquivo depois. Ver se tem como fazer isto

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class Mapeamento(BaseEstimator, TransformerMixin):    

    def __init__(self, variavel, mapa):
        #Parametros
        # -mapas -->lista com dicionários(mapas)
        self.variavel = variavel
        self.mapa = mapa 

    def fit(self, X, y=None):
        return self

    def transform(self, X):        
        X = X.copy()        
        X[self.variavel] = X[self.variavel].map(self.mapa)
        return X

## Carrega o pepeline treinado

In [18]:
model_pipe = joblib.load('model_pipe.joblib') 

## Gera a salva a nova tabela

In [48]:
# previsão
y_hat=model_pipe.predict(X)

# transformação dos dados
class_map = {0:'C', 1:'D', 2:'A', 3:'B' }
y_hat= pd.Series(y_hat).map(class_map)

#acrescenta na tabela
df['prev_val']=y_hat

#verificação
df.head()['prev_val']

0    B
1    C
2    A
3    B
4    A
Name: prev_val, dtype: object

In [52]:
# salva a nova tabela
df.to_csv("dados\\dados_test_prev_API.csv")

## Estatisticas com os valores previstos para a tabela de teste

As estatísticas abaixo servem para verificar se a aplicação do modelo com os dados de teste estão coerentes com a aplicação do modelo nos dados de treinamento e também para verificar se as previsão com o modelo implementado em um pacote estão funcionando corretamente

* Na construção do modelo os dados de treinamento foram divididos em dados de treinamento e de teste. Os dados de teste foram usados para o desenvolvimento do modelo. Usando estes dados de teste podemos verificar qual modelo fornece a melhor estatística. No caso deste trabalho a melhor estatistica aconteceu para o modelo "random tree" e isto resultou numa accuracy de 0.7290322580645161.
    Usando os dados de teste armazenados na tabela de teste o modelo forneceu uma accuracy de 0.7114427860696517. Com isto temso que as previsões para estes dados estão coerentes, pois forncem valores para a métrica semelhando ao btido no desenvolvimento
    
* Para o modelo implementado como pacote, serão feitos testes de previsão com o modelo e a méttrica dos teste deve apresentar os valor 0.7114427860696517 obtido neste notebook. Se isto acontecer o modelo foi reprozido perfeitamete no pacote


In [50]:
print('Valores previstos: ', y_hat[0:10].to_list())
print('Valores reais: ', y[0:10].to_list())
print("Acuracia:", metrics.accuracy_score(y, y_hat))

Valores previstos:  ['B', 'C', 'A', 'B', 'A', 'A', 'B', 'D', 'B', 'B']
Valores reais:  ['C', 'C', 'A', 'C', 'B', 'A', 'B', 'D', 'B', 'A']
Acuracia: 0.7114427860696517
